In [1]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from cloudant.client import  Cloudant
from werkzeug.utils import secure_filename
from flask import Flask, request, render_template, redirect, url_for,session


In [2]:
app=Flask(__name__)

client=Cloudant.iam('f17a994d-760b-40dc-baff-83fc3b995d7b-bluemix','69W0Zt5eGFE1LX4qtysIXCC4Xm-qG1l8ZyfpkpiS10oF',connect=True)
my_database = client['db']
app.secret_key="SECRET_KEY"
model=load_model(r"inception-diabetic.h5")


In [3]:


image_folder=os.path.join('static','images')
app.config['UPLOAD_FOLDER'] = image_folder


@app.route('/')
def index():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'drimage.jpg')
    return render_template('index.html',image=full_filename)

@app.route('/index')
def home():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'drimage.jpg')
    return render_template('index.html',image=full_filename)

@app.route('/register')
def register():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'registerimg.jpg')
    return render_template('register.html',image=full_filename)

@app.route('/afterreg',methods=['POST','GET'])
def afterreg():
    x=[x for x in request.form.values()]
    data={
        '_id':x[2],
        'name':x[0],
        'pwd':x[4],
        'email':x[1],
        'location':x[3],
        'securityquestion':x[5],
        'logintype':x[6]
    }
    query={'_id':{'$eq':data['_id']}}
    docs=my_database.get_query_result(query)
    if(len(docs.all())==0):
        url=my_database.create_document(data)
        full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'loginimg.jpg')
        return render_template('login.html',predict="Registration successfull please login using your credentials",image=full_filename)
    else:
        full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'registerimg.jpg')
        return render_template('register.html',pred="You are already a member login using your credentials",image=full_filename)

@app.route('/login')
def login():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'loginimg.jpg')
    return render_template('login.html',image=full_filename)

@app.route('/afterlogin', methods=['POST','GET'])
def afterlogin():
    user=request.form['phoneno']
    session['pn']=user
    passw=request.form['pwd']
    lgnas=request.form['loginas']
    
    query={'_id':{'$eq':user}}
    docs=my_database.get_query_result(query) 
    
    if(len(docs.all())==0):
        full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'loginimg.jpg')
        return render_template('login.html',predict="Phone number/id not found",image=full_filename)
    else:
        if((user==docs[0][0]['_id'] and passw==docs[0][0]['pwd'] and lgnas==docs[0][0]['logintype'] )):
            if(docs[0][0]['logintype']=='user'):
                full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'retina.jpg')
                full_filename1 = os.path.join(app.config['UPLOAD_FOLDER'], 'image6.png')
                return render_template('prediction.html',image=full_filename,image2=full_filename1)
            if(docs[0][0]['logintype']=='admin'):
                full_filename2 = os.path.join(app.config['UPLOAD_FOLDER'], 'adminimg.jpg')
                return render_template('admin.html',image=full_filename2)
        if(lgnas!=docs[0][0]['logintype']):
            full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'loginimg.jpg')
            return render_template('login.html',image=full_filename,predict="Incorrect Logintype")
        if(passw!=docs[0][0]['pwd']):
            full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'loginimg.jpg')
            return render_template('login.html',image=full_filename,predict="Incorrect password")

        
@app.route('/respond')
def respond():
    my_database_query = client['my_database_query']
    
    dt=[]
    for document in my_database_query:
        dt.append(document['who'])
        dt.append(document['phoneno'])
        dt.append(document['query'])
    return render_template('respond.html',data=dt)

@app.route('/afterrespond', methods=['POST','GET'])
def afterrespond():  
    my_database_query = client['my_database_query']
    x=[x for x in request.form.values()]
    data1={
        '_id':x[0],
        'who':x[1],
        'phoneno':x[2],
        'query':x[3]
    }
    query={'_id':{'$eq':data1['_id']}}
    docs=my_database_query.get_query_result(query)
    if(len(docs.all())==0):
        url=my_database_query.create_document(data1)
        my_database_query = client['my_database_query']
        dt=[]
        for document in my_database_query:
            dt.append(document['who'])
            dt.append(document['phoneno'])
            dt.append(document['query'])
        return render_template('respond.html',predict="Response posted Successfully",data=dt)
    else:
        url=my_database_query.create_document(data1)
        my_database_query = client['my_database_query']
        dt=[]
        for document in my_database_query:
            dt.append(document['who'])
            dt.append(document['phoneno'])
            dt.append(document['query'])
        return render_template('respond.html',predict="Response posted Successfully",data=dt)



@app.route('/fp')
def fp():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'],'forgotpw.png')
    return render_template('fp.html',image=full_filename)

@app.route('/afterfp', methods=['POST','GET'])
def afterfp():  
    pn=request.form['phoneno']
    securityques=request.form['secques']
    npassw=request.form['npwd']
    cpassw=request.form['cpwd']
    
    
    docs=my_database[pn]
    
    
    if(npassw==cpassw and securityques==docs['securityquestion']):
        full_filename = os.path.join(app.config['UPLOAD_FOLDER'],'loginimg.jpg')
        docs['pwd'] = cpassw
        docs.save() 
        return render_template('login.html',predict="Successfully updated",image=full_filename)
    if(securityques!=docs['securityquestion']):
        full_filename = os.path.join(app.config['UPLOAD_FOLDER'],'forgotpw.png')
        return render_template('fp.html',predict="Incorrect answer to security question",image=full_filename)
    if(npassw!=cpassw):
        full_filename = os.path.join(app.config['UPLOAD_FOLDER'],'forgotpw.png')
        return render_template('fp.html',predict="New and confirm password does not match",image=full_filename)
    
    

@app.route('/prediction')
def prediction():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'retina.jpg')
    full_filename1 = os.path.join(app.config['UPLOAD_FOLDER'], 'image6.png')
    return render_template('prediction.html',image=full_filename,image2=full_filename1)

@app.route('/afterpred',methods=["GET","POST"])
def aftepred():
    if request.method=="POST":
        full_filename2 = os.path.join(app.config['UPLOAD_FOLDER'], 'retina.jpg')
        full_filename1 = os.path.join(app.config['UPLOAD_FOLDER'], 'image6.png')
        f=request.files['pfile']
        print("yes")
        filepath=os.path.join('static','uploads',f.filename)
        f.save(filepath)
        print("saved")
        img=image.load_img(filepath,target_size=(224,224))
        x=image.img_to_array(img)
        x=np.expand_dims(x,axis=0)
        img_data=preprocess_input(x)
        prediction=np.argmax(model.predict(img_data),axis=1)
        index=["no dr","mild dr","moderate dr","severe dr","proliferate"]
        result=str(index[prediction[0]])
        print(result)
        return render_template('prediction.html',prediction=result,image=full_filename2,image2=full_filename1)
    else:
        full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'loginimg.jpg')
        return render_template('login.html',pred="Please login using your credentials",image=full_filename)


@app.route('/query')
def query():
    my_database_query = client['my_database_query']
    
    dt=[]
    for document in my_database_query:
        dt.append(document['who'])
        dt.append(document['phoneno'])
        dt.append(document['query'])
    return render_template('query1.html',data=dt)

@app.route('/afterquery', methods=['POST','GET'])
def afterquery():  
    my_database_query = client['my_database_query']
    x=[x for x in request.form.values()]
    data1={
        '_id':x[0],
        'who':x[1],
        'phoneno':x[2],
        'query':x[3],
    }
    query={'_id':{'$eq':data1['_id']}}
    docs=my_database_query.get_query_result(query)
    if(len(docs.all())==0):
        url=my_database_query.create_document(data1)
        my_database_query = client['my_database_query']
        dt=[]
        for document in my_database_query:
            dt.append(document['who'])
            dt.append(document['phoneno'])
            dt.append(document['query'])
        return render_template('query1.html',predict="Query submitted Successfully",data=dt)
    else:
        url=my_database_query.create_document(data1)
        my_database_query = client['my_database_query']
        dt=[]
        for document in my_database_query:
            dt.append(document['who'])
            dt.append(document['phoneno'])
            dt.append(document['query'])
        return render_template('query1.html',predict="Query submitted Successfully",data=dt)

@app.route('/admin')
def admin():
    full_filename2 = os.path.join(app.config['UPLOAD_FOLDER'], 'adminimg.png')
    return render_template('admin.html',image=full_filename2)

@app.route('/locsugges')
def locsugess():
    print("location")
    dbl=client['db1']
    dbu=client['db']
    print("good")
    pn=session['pn']
    doc1=dbu[pn]
    location=doc1["location"]
    print("new")
    docs=dbl[location.upper()]
    hospital=[]
    location=[]
    for h in range(len(docs["hospitals"])):
        hospital.append(docs['hospitals'][h])
    for l in range(len(docs["locations"])):
        location.append(docs['locations'][l])
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'])
    return render_template('locsugges.html',hospital=hospital,location=location)
@app.route('/uploc')
def uploc():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'])
    return render_template('uploc.html')

@app.route('/afteruploc',methods=["GET","POST"])
def afteruploc():
    loc=request.form['loch']
    hname=request.form['hname']
    lname=request.form['lname']
    contact=request.form['contact']
    dbl=client['db1']
    docs=dbl[loc.upper()]
    hn=hname.strip().upper()
    count=0
    for i in range(len(docs['hospitals'])):
       if docs['hospitals'][i].strip().upper()==hn:
          docs['contacts'][i]=contact
          docs['locations'][i]=lname
          count=1
          docs.save()
          break
    if(count==0):
        docs['hospitals'].append(hn)
        docs['contacts'].append(contact)
        docs.save()
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'])
    return render_template('uploc.html',predict="Updated or added successfully")


@app.route('/modify',methods=["GET","POST"])
def modify():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'])
    return render_template('modify.html',predict="Please enter your new location")

@app.route('/aftermodify',methods=["GET","POST"])
def aftermodify():
    if request.method=="POST":
        user=session['pn']
        pwd=request.form['pwd']
        nloc=request.form['location']
        docs=my_database[user]
        if (pwd==docs['pwd']):
            full_filename = os.path.join(app.config['UPLOAD_FOLDER'],'pimg.jpg')
            docs['location'] = nloc
            docs.save()
            return render_template('modify.html',predict="Successfully updated")
        else:
           full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'retina.jpg')
           full_filename1 = os.path.join(app.config['UPLOAD_FOLDER'], 'image6.png')
           return render_template('prediction.html',image=full_filename,image2=full_filename1)



@app.route('/logout')
def logout():
    session.pop('pn', None)
    return render_template('logout.html',)

if __name__=="__main__":
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Nov/2022 19:26:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2022 19:26:10] "GET /static/images/drimage.jpg HTTP/1.1" 304 -
127.0.0.1 - - [14/Nov/2022 19:26:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Nov/2022 19:26:18] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2022 19:26:18] "GET /static/images/loginimg.jpg HTTP/1.1" 304 -
127.0.0.1 - - [14/Nov/2022 19:26:31] "POST /afterlogin HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2022 19:26:31] "GET /static/images/retina.jpg HTTP/1.1" 304 -
127.0.0.1 - - [14/Nov/2022 19:26:31] "GET /static/images/image6.png HTTP/1.1" 304 -


location
good
new


[2022-11-14 19:26:36,441] ERROR in app: Exception on /locsugges [GET]
Traceback (most recent call last):
  File "c:\Users\Vetri\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Vetri\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Vetri\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Vetri\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\Vetri\AppData\Local\Temp\ipykernel_61608\663466408.py", line 242, in locsugess
    hospital=hospital.append(docs['hospitals'][h])
AttributeError: 'NoneType' object has no attr

location
good
new


127.0.0.1 - - [14/Nov/2022 19:27:33] "GET /modify HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2022 19:27:39] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2022 19:27:39] "GET /static/images/drimage.jpg HTTP/1.1" 304 -
127.0.0.1 - - [14/Nov/2022 19:27:42] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2022 19:27:42] "GET /static/images/loginimg.jpg HTTP/1.1" 304 -
127.0.0.1 - - [14/Nov/2022 19:27:57] "POST /afterlogin HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2022 19:27:58] "GET /static/images/adminimg.jpg HTTP/1.1" 404 -
